In [ ]:
import pandas as pd

In [11]:
import pandas as pd
from datetime import datetime
import os
import numpy as np

In [4]:
weather_files = [x for x in os.listdir('weather') if x.endswith('.csv')]

In [26]:
df: pd.DataFrame
df = pd.read_csv(os.path.join('weather', weather_files[0]), encoding='euc-kr')
df = df[['일시', '습도', '강수량', '최고 기온', '최저 기온']]
df.columns = ['date', 'humidity', 'rain', 'tmax', 'tmin']

df["time"] = df["date"].map(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M").strftime("%Y%m%d%H%M"))
df['date'] = df['date'].map(lambda x: datetime.strptime(x[:len("2018-01-01")], "%Y-%m-%d"))
df = df.replace(-99.8, np.NAN)
df = df.replace(-999.0, np.NAN)
df

0       201701010300
1       201701010600
2       201701010900
3       201701011200
4       201701011500
            ...     
2906    201712301200
2907    201712301500
2908    201712301800
2909    201712302100
2910    201712310000
Name: time, Length: 2911, dtype: object


,date,humidity,rain,tmax,tmin,time
0,2017-01-01,62.3,NaN,NaN,NaN,201701010300
1,2017-01-01,61.7,NaN,NaN,NaN,201701010600
2,2017-01-01,67.0,NaN,-1.1,NaN,201701010900
3,2017-01-01,71.5,NaN,NaN,NaN,201701011200
4,2017-01-01,76.7,NaN,NaN,NaN,201701011500
...,...,...,...,...,...,...
2906,2017-12-30,59.4,NaN,NaN,NaN,201712301200
2907,2017-12-30,63.5,NaN,NaN,NaN,201712301500
2908,2017-12-30,72.6,NaN,NaN,NaN,201712301800
2909,2017-12-30,89.2,0.7,NaN,-13.2,201712302100


70.91428571428571 -5.0 -1.1 0.0
77.0375 -8.3 0.6 0.0
70.8875 -8.6 -1.5 0.3
70.5625 -10.15 -5.6 0.0
64.42500000000001 -13.75 -7.7 0.0
53.45 -7.2 -4.0 0.0
54.275 -10.4 -2.6 0.0
78.875 -8.7 -3.0 2.0
46.9375 -11.8 -3.4 0.2
42.7875 -14.7 -10.8 0.0
44.8625 -17.4 -11.9 0.0
42.75 -17.7 -12.1 0.0
64.05000000000001 -21.0 -13.2 0.0
63.662499999999994 -16.5 -10.2 0.0
72.1625 -10.2 -8.2 0.0
77.775 -9.4 -4.8 0.0
69.4125 -11.3 -3.0 0.0
67.5375 -11.5 -5.2 0.0
68.125 -13.6 -7.7 0.0
70.0625 -16.049999999999997 -12.0 0.0
71.875 -18.1 -12.3 0.0
74.4125 -18.700000000000003 -14.0 0.0
49.8125 -16.7 -12.2 0.0
47.787499999999994 -14.899999999999999 -9.0 0.0
40.4125 -8.4 -6.7 0.0
72.8125 -8.2 -0.3 0.0
37.775000000000006 -8.3 -5.5 0.0
52.650000000000006 -8.55 -2.1 0.4
63.8875 -13.35 -8.3 0.0
46.5375 -14.600000000000001 -9.1 0.0
60.8625 -10.7 -4.9 0.0
37.324999999999996 -12.149999999999999 -9.3 0.0
38.175 -7.5 -2.9 0.0
38.800000000000004 -5.75 0.3 0.0
43.55 -6.1 1.6 0.0
62.8875 -8.15 -2.3 0.0
44.5875 -11.35 -6.5 

In [28]:
import math
from datetime import date

def get_ra(site: pd.Series, time: pd.Series):  # Extract Location-Based RA from Site and Time; Returns Series
    def cos(x):  # element-wise override
        if type(x) in (int, float):
            return math.cos(x)
        elif type(x) is pd.Series:
            x: pd.Series
            return x.map(math.cos)

    def sin(x):
        if type(x) in (int, float):
            return math.sin(x)
        elif type(x) is pd.Series:
            x: pd.Series
            return x.map(math.sin)

    COORDINATE = {
        'IT-Cas': (45.07004722, 8.717522222),
        'JP-Mse': (36.05393, 140.02693),
        'KR-CRK': (38.2013, 127.2506),
        'PH-RiF': (14.14119, 121.26526),
        'US-HRA': (34.585208, -91.751735),
        'US-HRC': (34.58883, -91.751658),
        'US-Twt': (38.1087204, -121.6531),
        'Ru-Bld': (43.13, 131.9),
        'FNL': (0, 0),
        'GRK': (0, 0),
        'CFK': (0, 0)  # TODO: Site 정보 추가
    }

    coord = site.map(lambda x: COORDINATE[x])
    coord_lat = coord.map(lambda x: x[0] * math.pi / 180)

    DOY = time.map(lambda ts: date(int(str(ts)[0:4]), int(str(ts)[4:6]), int(str(ts)[6:8])).timetuple().tm_yday)
    HOUR = time.map(lambda ts: int(str(ts)[-4:-2]))

    Ll = 0
    Ls = 0

    B = 360 * (DOY - 81) / 365 * math.pi / 180
    ET = 9.87 * sin(2 * B) - 7.53 * cos(B) - 1.5 * sin(B)
    ST = HOUR + ET / 60 + 4 / 60 * (Ll - Ls)
    hangle = (ST - 12) * math.pi / 12

    ST0 = HOUR - 1 + ET / 60 + 4 / 60 * (Ll - Ls)
    hangle0 = (ST0 - 12) * math.pi / 12

    DEC = -23.45 * cos(360 / 365 * (DOY + 10) * math.pi / 180) * math.pi / 180
    ISC = -1367 * 3600 / 1000000
    E0 = -1 + 0.0033 * cos(2 * math.pi * DOY / 365)

    Ra_hourly = (12 * 3.6 / math.pi) * ISC * E0 * (
            ((sin(coord_lat) * cos(DEC)) * (sin(hangle) - sin(hangle0))) + (math.pi / 180 * (hangle - hangle0)) *
            (sin(coord_lat) * sin(DEC)))

    return Ra_hourly

In [29]:
site = 'Ru-Bld'
df['site'] = site

get_ra(df['site'], df['time'])


0       -8.979077
1       -1.705767
2        6.518472
3       10.875992
4        8.814218
          ...    
2906    10.870546
2907     8.778578
2908     1.495679
2909    -6.711929
2910   -11.038339
Length: 2911, dtype: float64